# Transfer Learning with TensorFlow Part 2: Fine Tuning

In the previous notebook, we covered transfer learning *feature extraction*, now it's time to learn about a new kind of transfer learning: *fine-tuning*

In [ ]:
# Hello